### GraphSVX

Here we visualize the explainer results and analyze specific graph theoretic properties for datatset ```syn1```

In [1]:
# imports
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
results = '../results/node_masks/'
# Load produced masks
dirs = os.listdir(results)
dirs

['node_mask_syn1_explain_node_400.npy',
 'node_mask_syn1_explain_node_405.npy',
 'node_mask_syn1_explain_node_410.npy',
 'node_mask_syn1_explain_node_415.npy',
 'node_mask_syn1_explain_node_420.npy',
 'node_mask_syn1_explain_node_425.npy',
 'node_mask_syn1_explain_node_430.npy',
 'node_mask_syn1_explain_node_435.npy',
 'node_mask_syn1_explain_node_440.npy',
 'node_mask_syn1_explain_node_445.npy',
 'node_mask_syn1_explain_node_450.npy',
 'node_mask_syn1_explain_node_455.npy',
 'node_mask_syn1_explain_node_460.npy',
 'node_mask_syn1_explain_node_465.npy',
 'node_mask_syn1_explain_node_470.npy',
 'node_mask_syn1_explain_node_475.npy',
 'node_mask_syn1_explain_node_480.npy',
 'node_mask_syn1_explain_node_485.npy',
 'node_mask_syn1_explain_node_490.npy',
 'node_mask_syn1_explain_node_495.npy',
 'node_mask_syn1_explain_node_500.npy',
 'node_mask_syn1_explain_node_505.npy',
 'node_mask_syn1_explain_node_510.npy',
 'node_mask_syn1_explain_node_515.npy',
 'node_mask_syn1_explain_node_520.npy',


In [3]:
# Get the masks
node_masks = []
# This would print all the files and directories
for file in dirs:
    # Check if file extension is ".npy" which are
    # numpy binary files to represent large data
    if file.split('.')[-1] == 'npy' and file.split('_')[2] == 'syn1':
        # print(file)
        node_masks.append(file)
len(node_masks)

60

In [4]:
masked_subgraph = '../results/exp_mask_adj/'

# Load produced masks
dirs = os.listdir(masked_subgraph)
dirs

['masked_adj_syn1_explain_node_400.npy',
 'masked_adj_syn1_explain_node_405.npy',
 'masked_adj_syn1_explain_node_410.npy',
 'masked_adj_syn1_explain_node_415.npy',
 'masked_adj_syn1_explain_node_420.npy',
 'masked_adj_syn1_explain_node_425.npy',
 'masked_adj_syn1_explain_node_430.npy',
 'masked_adj_syn1_explain_node_435.npy',
 'masked_adj_syn1_explain_node_440.npy',
 'masked_adj_syn1_explain_node_445.npy',
 'masked_adj_syn1_explain_node_450.npy',
 'masked_adj_syn1_explain_node_455.npy',
 'masked_adj_syn1_explain_node_460.npy',
 'masked_adj_syn1_explain_node_465.npy',
 'masked_adj_syn1_explain_node_470.npy',
 'masked_adj_syn1_explain_node_475.npy',
 'masked_adj_syn1_explain_node_480.npy',
 'masked_adj_syn1_explain_node_485.npy',
 'masked_adj_syn1_explain_node_490.npy',
 'masked_adj_syn1_explain_node_495.npy',
 'masked_adj_syn1_explain_node_500.npy',
 'masked_adj_syn1_explain_node_505.npy',
 'masked_adj_syn1_explain_node_510.npy',
 'masked_adj_syn1_explain_node_515.npy',
 'masked_adj_syn

In [5]:
# Get the masks
mask_adj = []
# This would print all the files and directories
for file in dirs:
    # Check if file extension is ".npy" which are
    # numpy binary files to represent large data
    if file.split('.')[-1] == 'npy' and file.split('_')[2] == 'syn1':
        # print(file)
        mask_adj.append(file)
len(mask_adj)

60

In [6]:
def show_adjacency_full(path,mask):
    """Obtain the numpy array of the mask

    Args:
        mask (str): Filename containing the mask
        ax (Axes class object, optional): Axis. Defaults to None.

    Returns:
        numpy array: Numpy array version of the mask
    """
    # Obtain mask as array from the filename sent in var "mask"
    adj = np.load(os.path.join(path, mask), allow_pickle=True)

    return adj

### Analyzing Graph Properties

Here we measure the following quantities:

- Nodes: |V|
- Edges: |E|
- Avg degree
- Diameter of the graph
- Sparsity measure: edge density = |E|/C(|V|,2)
- Node Betweenness centrality

In [7]:
# Get data adjacency matrix
datadir = '../results/dataset_matrix/'
matrices = os.listdir(datadir)
matrices

['adj_matrix_syn1.npy']

In [8]:
data = []
dataset = 'syn1.npy'
# This would print all the files and directories
for file in matrices:
    # Check if file extension is ".npy" which are
    # numpy binary files to represent large data
    if file.split('.')[-1] == 'npy' and file.split('_')[2] == dataset:
        data.append(file)
data

['adj_matrix_syn1.npy']

In [9]:
adj_matrix = np.load(os.path.join(datadir, data[0]), allow_pickle=True)
adj_matrix.shape

(700, 700)

In [25]:
def normalize(A):
    scale_factor = A.max() - A.min()
    B = np.ones_like(A)*A.min()

    A = (A - B)/scale_factor
    return A

def centrality(G,mask,node_to_explain):
    
    print(f"Node #{node_to_explain}: {list(G.nodes())}")
    sources = [node_to_explain]
    targets = [v for v in list(G.nodes()) if v!=node_to_explain]
    bc = nx.betweenness_centrality_subset(G,sources,targets)

    # By explainer
    A = mask.copy()
    
    # Importance from bc
    B = np.zeros_like(A)
    for v in list(bc.keys()):
        B[v] = bc.get(v)
    
    A1 = normalize(A)
    B1 = normalize(B)

    if(np.equal(A1,B1) == False):
        return 1/(np.linalg.norm(A1 - B1, 2)**2)
    else:
        return 1

def graph_prop(graph,mask,node_to_explain):

    v = graph.number_of_nodes()
    e = graph.number_of_edges()
    avg_degree = float('%.3f'%(2*e/v))
    diameter = nx.diameter(graph)
    sparsity = float('%.3f'%(2*e/(v*(v-1))))
    coh = centrality(graph,mask,node_to_explain)

    return [v,e,avg_degree,diameter,sparsity,coh]

In [11]:
def get_node_to_explain(filename):

    required_slice = filename.split('_')[-1]
    node = int(required_slice.split('.')[0])

    return node

In [29]:
graph_properties = []
dataset_G = nx.from_numpy_array(adj_matrix)

# len(mask_adj)
for i in range(len(mask_adj)):
    # adj = show_adjacency_full(masked_subgraph,mask_adj[i])  # Get the adjacency matrix of the explanation
    # G = nx.from_numpy_array(adj)    # Get the explanation subgraph
    node_mask = show_adjacency_full(results, node_masks[i])  # Get the node mask for the considered explanation
    idxs = np.where(node_mask != 0)[0]
    G = nx.induced_subgraph(dataset_G, idxs)

    node_to_explain = get_node_to_explain(mask_adj[i])  # Get the label of the node being explained

    print(f"Analyzing graph properties for explanation of node #{node_to_explain}")
    # graph_properties.append(graph_prop(G,node_mask,node_to_explain))
    print(f"Number of connected components of G: {nx.number_connected_components(G)}")

print("All explanations successfully analyzed!")

Analyzing graph properties for explanation of node #400
Number of connected components of G: 2
Analyzing graph properties for explanation of node #405
Number of connected components of G: 1
Analyzing graph properties for explanation of node #410
Number of connected components of G: 2
Analyzing graph properties for explanation of node #415
Number of connected components of G: 2
Analyzing graph properties for explanation of node #420
Number of connected components of G: 2
Analyzing graph properties for explanation of node #425
Number of connected components of G: 2
Analyzing graph properties for explanation of node #430
Number of connected components of G: 2
Analyzing graph properties for explanation of node #435
Number of connected components of G: 2
Analyzing graph properties for explanation of node #440
Number of connected components of G: 2
Analyzing graph properties for explanation of node #445
Number of connected components of G: 2
Analyzing graph properties for explanation of node